In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ju"

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
running_order.head()

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,28,OLV Baselland 1,OLV Baselland,SUI,1,10.9,timon schweizer,1,Timon Schweizer,4.0,1.935,0.129,4.281278,1.927112,1.960336,0.016612,0.100000,1.935,0.129
1,1,28,OLV Baselland 1,OLV Baselland,SUI,2,10.5,kaspar hägler,2,Kaspar Hägler,2.0,1.842,0.051,6.382389,1.775996,1.914535,0.069270,0.100000,1.842,0.100
2,2,28,OLV Baselland 1,OLV Baselland,SUI,3,13.2,fabian hertner,3,Fabian Hertner,6.0,1.746,0.070,6.347555,1.781784,1.980862,0.099539,0.100000,1.746,0.070
3,3,28,OLV Baselland 1,OLV Baselland,SUI,4,7.3,jannis schönleber,4,Jannis Schönleber,2.0,1.949,0.118,6.251983,1.908036,2.094361,0.093162,0.100000,1.949,0.100
4,4,28,OLV Baselland 1,OLV Baselland,SUI,5,7.8,stefan lombriser,5,Stefan Lombriser,4.0,2.007,0.110,6.963056,1.826875,2.038693,0.105909,0.105909,2.007,0.110


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,28,OLV Baselland 1,1,10.9,Timon Schweizer,1.935,0.129
1,28,OLV Baselland 1,2,10.5,Kaspar Hägler,1.842,0.100
2,28,OLV Baselland 1,3,13.2,Fabian Hertner,1.746,0.070
3,28,OLV Baselland 1,4,7.3,Jannis Schönleber,1.949,0.100
4,28,OLV Baselland 1,5,7.8,Stefan Lombriser,2.007,0.110


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                           \
leg                            1                        2   
team_id                                                     
28               OLV Baselland 1          OLV Baselland 1   
36          IK Hakarpspojkarna 1     IK Hakarpspojkarna 1   
60            Suunta Jyväskylä 1       Suunta Jyväskylä 1   
87       Kouvolan Suunnistajat 1  Kouvolan Suunnistajat 1   
96               OLV Baselland 2          OLV Baselland 2   

                                                           \
leg                            3                        4   
team_id                                                     
28               OLV Baselland 1          OLV Baselland 1   
36          IK Hakarpspojkarna 1     IK Hakarpspojkarna 1   
60            Suunta Jyväskylä 1       Suunta Jyväskylä 1   
87       Kouvolan Suunnistajat 1  Kouvolan Suunnistajat 1   
96               OLV Baselland 2          OLV Baselland 2   

                                                           \
leg                            5                        6   
team_id                                                     
28               OLV Baselland 1          OLV Baselland 1   
36          IK Hakarpspojkarna 1     IK Hakarpspojkarna 1   
60            Suunta Jyväskylä 1       Suunta Jyväskylä 1   
87       Kouvolan Suunnistajat 1  Kouvolan Suunnistajat 1   
96               OLV Baselland 2          OLV Baselland 2   

                                  dist              ... log_mean         \
leg                            7     1     2     3  ...        5      6   
team_id                                             ...                   
28               OLV Baselland 1  10.9  10.5  13.2  ...  2.00700  1.879   
36          IK Hakarpspojkarna 1  10.9  10.5  13.2  ...  1.99300  1.978   
60            Suunta Jyväskylä 1  10.9  10.5  13.2  ...  2.11800  2.064   
87       Kouvolan Suunnistajat 1  10.9  10.5  13.2  ...  2.00100  1.986   
96               OLV Baselland 2  10.9  10.5  13.2  ...  1.98856  2.107   

                    log_std                                                  
leg             7         1      2      3         4      5         6      7  
team_id                                                                      
28       1.850000  0.129000  0.100  0.070  0.100000  0.110  0.091000  0.100  
36       1.838000  0.131000  0.095  0.081  0.079000  0.117  0.099000  0.083  
60       1.925000  0.092000  0.081  0.102  0.123000  0.102  0.118000  0.090  
87       1.945000  0.100000  0.098  0.080  0.084000  0.108  0.093000  0.112  
96       1.676371  0.114143  0.100  0.100  0.149311  0.100  0.117543  0.100  

[5 rows x 35 columns]

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(471, 35)

(455, 35)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict, **fin_sum_log_means_dict, **fin_sum_log_stds_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_sum_log_mean_5,fin_sum_log_mean_6,fin_sum_log_mean_7,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4,fin_sum_log_std_5,fin_sum_log_std_6,fin_sum_log_std_7
team_id,,,,,,,,,,,,,,,,,,,,,
28,58.376902,121.005292,194.795530,243.915460,299.276038,369.370899,447.637588,97.023164,167.825337,245.875402,...,5.793425,5.994253,6.181027,0.129341,0.083691,0.059775,0.051979,0.047198,0.042060,0.038739
36,53.934071,119.038980,201.187719,246.997753,301.053981,378.134473,456.479496,89.891461,163.663720,253.039342,...,5.798750,6.017643,6.198231,0.131591,0.081364,0.059189,0.050714,0.046718,0.042298,0.037907
60,61.692444,129.973332,225.009374,284.809681,347.480380,429.487747,515.543623,88.605108,165.715262,280.155049,...,5.941511,6.149592,6.323183,0.092155,0.061622,0.055538,0.051002,0.045812,0.043436,0.039296
87,69.856854,140.681950,233.367055,284.635165,340.165707,417.866114,503.910205,103.135001,184.952943,288.060762,...,5.912091,6.110634,6.293798,0.099885,0.069820,0.053393,0.046494,0.042586,0.038622,0.036957
96,58.605919,140.184563,246.090880,294.004430,349.750301,435.731699,502.992495,91.762098,188.818792,311.943821,...,5.953870,6.168117,6.303776,0.113589,0.075929,0.060755,0.056378,0.050324,0.046660,0.042517


In [8]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()

,team_id,team_1,team_2,team_3,team_4,team_5,team_6,team_7,dist_1,dist_2,...,log_mean_5,log_mean_6,log_mean_7,log_std_1,log_std_2,log_std_3,log_std_4,log_std_5,log_std_6,log_std_7
0,28,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,10.9,10.5,...,2.007000,1.879000,1.850000,0.129000,0.100000,0.070000,0.100000,0.110000,0.091000,0.100000
1,36,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,10.9,10.5,...,1.993000,1.978000,1.838000,0.131000,0.095000,0.081000,0.079000,0.117000,0.099000,0.083000
2,60,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,10.9,10.5,...,2.118000,2.064000,1.925000,0.092000,0.081000,0.102000,0.123000,0.102000,0.118000,0.090000
3,87,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,10.9,10.5,...,2.001000,1.986000,1.945000,0.100000,0.098000,0.080000,0.084000,0.108000,0.093000,0.112000
4,96,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,10.9,10.5,...,1.988560,2.107000,1.676371,0.114143,0.100000,0.100000,0.149311,0.100000,0.117543,0.100000
5,97,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,10.9,10.5,...,1.935000,1.874000,2.037000,0.135117,0.116000,0.098000,0.139937,0.124000,0.086000,0.100000
6,104,Pellon Ponsi 1,Pellon Ponsi 1,Pellon Ponsi 1,Pellon Ponsi 1,Pellon Ponsi 1,Pellon Ponsi 1,Pellon Ponsi 1,10.9,10.5,...,2.142000,1.896000,1.870000,0.092000,0.084000,0.080000,0.112000,0.104000,0.109000,0.078000
7,138,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,10.9,10.5,...,2.325000,2.274000,1.817044,0.122788,0.100000,0.100000,0.150990,0.167585,0.122889,0.100000
8,140,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,10.9,10.5,...,2.273000,2.100000,2.165000,0.085000,0.114766,0.105000,0.158767,0.138721,0.108000,0.116000
9,143,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,10.9,10.5,...,2.227000,2.013000,2.359407,0.138575,0.194749,0.165226,0.235287,0.200753,0.224190,0.181328


In [9]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [10]:
estimates.head()

,team_1,team_2,team_3,team_4,team_5,team_6,team_7,dist_1,dist_2,dist_3,...,fin_sum_log_mean_5,fin_sum_log_mean_6,fin_sum_log_mean_7,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4,fin_sum_log_std_5,fin_sum_log_std_6,fin_sum_log_std_7
team_id,,,,,,,,,,,,,,,,,,,,,
28,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,10.9,10.5,13.2,...,5.793425,5.994253,6.181027,0.129341,0.083691,0.059775,0.051979,0.047198,0.042060,0.038739
36,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,10.9,10.5,13.2,...,5.798750,6.017643,6.198231,0.131591,0.081364,0.059189,0.050714,0.046718,0.042298,0.037907
60,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,10.9,10.5,13.2,...,5.941511,6.149592,6.323183,0.092155,0.061622,0.055538,0.051002,0.045812,0.043436,0.039296
87,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,10.9,10.5,13.2,...,5.912091,6.110634,6.293798,0.099885,0.069820,0.053393,0.046494,0.042586,0.038622,0.036957
96,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,10.9,10.5,13.2,...,5.953870,6.168117,6.303776,0.113589,0.075929,0.060755,0.056378,0.050324,0.046660,0.042517


In [11]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist               name  log_mean  log_std  \
team_id leg                                                                
28      1    OLV Baselland 1  10.9    Timon Schweizer     1.935    0.129   
        2    OLV Baselland 1  10.5      Kaspar Hägler     1.842    0.100   
        3    OLV Baselland 1  13.2     Fabian Hertner     1.746    0.070   
        4    OLV Baselland 1   7.3  Jannis Schönleber     1.949    0.100   
        5    OLV Baselland 1   7.8   Stefan Lombriser     2.007    0.110   
        6    OLV Baselland 1  11.1       Noah Zbinden     1.879    0.091   
        7    OLV Baselland 1  12.9    Christoph Meier     1.850    0.100   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
28      1      58.376902   97.023164   75.319480   76.042913   
        2     121.005292  167.825337  142.203563  142.691081   
        3     194.795530  245.875402  218.123307  218.603584   
        4     243.915460  299.116091  269.678341  270.135608   
        5     299.276038  360.342159  328.010291  328.500946   
        6     369.370899  435.803012  401.179023  401.471944   
        7     447.637588  521.926920  483.653736  483.850996   

             fin_sum_log_mean  fin_sum_log_std  
team_id leg                                     
28      1            4.322931         0.129341  
        2            4.957175         0.083691  
        3            5.385471         0.059775  
        4            5.597572         0.051979  
        5            5.793425         0.047198  
        6            5.994253         0.042060  
        7            6.181027         0.038739

In [12]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95"]]

name  fin_start95  fin_median   fin_end95
team_id leg                                                         
530     1        Mikko Peltonen    82.540967   99.408136  120.893996
        2         Jeppe Koivula   159.085679  180.623890  205.719617
        3          Joni Freeman   265.437894  296.865821  332.222990
        4       Pasi Huhtiniemi   328.357531  362.600782  400.438747
        5          Tommi Pajala   393.204512  431.437472  473.715579
        6         Olavi Kanerva   492.492697  541.822215  596.972593
        7       Janne Vaittinen   613.187936  676.900998  751.573441
1201    1         Jyri Kytömäki    83.227684  106.907487  137.108953
        2             Mikko Apo   183.143985  219.065751  262.022877
        3           Aki Kaivola   315.837913  360.445655  410.907350
        4            Mika Tuoma   385.371017  435.622688  491.956844
        5    Oskari Pirttikoski   481.210497  539.792435  605.054557
        6            Pasi Koski   603.707634  669.450318  742.367170
        7    Antti-Ville Jokela   725.879385  798.132558  876.306170

In [13]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates.head()

team  dist               name  log_mean  log_std  \
team_id leg                                                                
28      1    OLV Baselland 1  10.9    Timon Schweizer     1.935    0.129   
        2    OLV Baselland 1  10.5      Kaspar Hägler     1.842    0.100   
        3    OLV Baselland 1  13.2     Fabian Hertner     1.746    0.070   
        4    OLV Baselland 1   7.3  Jannis Schönleber     1.949    0.100   
        5    OLV Baselland 1   7.8   Stefan Lombriser     2.007    0.110   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
28      1      58.376902   97.023164   75.319480   76.042913   
        2     121.005292  167.825337  142.203563  142.691081   
        3     194.795530  245.875402  218.123307  218.603584   
        4     243.915460  299.116091  269.678341  270.135608   
        5     299.276038  360.342159  328.010291  328.500946   

             fin_sum_log_mean  fin_sum_log_std               fin_time_median  \
team_id leg                                                                    
28      1            4.322931         0.129341 2019-06-16 00:15:19.168772221   
        2            4.957175         0.083691 2019-06-16 01:22:12.213791132   
        3            5.385471         0.059775 2019-06-16 02:38:07.398442745   
        4            5.597572         0.051979 2019-06-16 03:29:40.700456381   
        5            5.793425         0.047198 2019-06-16 04:28:00.617483854   

                         fin_time_start95                fin_time_end95  
team_id leg                                                              
28      1   2019-06-15 23:58:22.614108801 2019-06-16 00:37:01.389866829  
        2   2019-06-16 01:01:00.317515135 2019-06-16 01:47:49.520224333  
        3   2019-06-16 02:14:47.731823921 2019-06-16 03:05:52.524143934  
        4   2019-06-16 03:03:54.927622080 2019-06-16 03:59:06.965449333  
        5   2019-06-16 03:59:16.562303305 2019-06-16 05:00:20.529564857

In [14]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["fin_time_median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["fin_time_start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["fin_time_end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")
for_print = for_print[['team',
 'leg',
 'name', 
 'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_print[for_print['team'].str.contains("Reak")]

,team,leg,name,fin_time_start95,fin_time_end95,fin_time_median
team_id,,,,,,
530,Reaktor Innovations 1,1,Mikko Peltonen,00:22,01:00,00:39
530,Reaktor Innovations 1,2,Jeppe Koivula,01:39,02:25,02:00
530,Reaktor Innovations 1,3,Joni Freeman,03:25,04:32,03:56
530,Reaktor Innovations 1,4,Pasi Huhtiniemi,04:28,05:40,05:02
530,Reaktor Innovations 1,5,Tommi Pajala,05:33,06:53,06:11
530,Reaktor Innovations 1,6,Olavi Kanerva,07:12,08:56,08:01
530,Reaktor Innovations 1,7,Janne Vaittinen,09:13,11:31,10:16
1201,Reaktor Innovations 2,1,Jyri Kytömäki,00:23,01:17,00:46
1201,Reaktor Innovations 2,2,Mikko Apo,02:03,03:22,02:39


In [15]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median
420,483,1,Rasti-Jyry 1,Heikki Alanko,2.185,0.115000,4.573999,0.115156,2019-06-15 21:17:11.944720030+00:00,2019-06-15 22:01:10.207587957+00:00,2019-06-15 21:36:53.459855080+00:00
421,483,2,Rasti-Jyry 1,Atte Honkonen,2.071,0.103000,5.197217,0.079076,2019-06-15 22:34:32.659025192+00:00,2019-06-15 23:30:35.928379297+00:00,2019-06-15 23:00:40.689081907+00:00
422,483,3,Rasti-Jyry 1,Tuomas Maisala,2.168,0.112000,5.693252,0.064851,2019-06-16 00:21:40.335225582+00:00,2019-06-16 01:37:02.975294590+00:00,2019-06-16 00:57:01.111892462+00:00
423,483,4,Rasti-Jyry 1,Lauri Tapola,2.243,0.118000,5.903041,0.057480,2019-06-16 01:27:10.221459866+00:00,2019-06-16 02:49:26.756425619+00:00,2019-06-16 02:06:11.293739319+00:00
424,483,5,Rasti-Jyry 1,Jussi-Pekka Hanhela,2.233,0.154419,6.085930,0.054354,2019-06-16 02:35:08.009869576+00:00,2019-06-16 04:07:39.227734327+00:00,2019-06-16 03:19:49.973032475+00:00
425,483,6,Rasti-Jyry 1,Juha-Pekka Strömberg,2.334,0.140000,6.319580,0.051995,2019-06-16 04:22:07.063197613+00:00,2019-06-16 06:14:27.651724815+00:00,2019-06-16 05:15:28.443272591+00:00
426,483,7,Rasti-Jyry 1,Ville Glad,2.126,0.112000,6.498570,0.047165,2019-06-16 06:05:58.693776608+00:00,2019-06-16 08:07:22.679134369+00:00,2019-06-16 07:04:09.046822309+00:00
1750,1297,1,Rasti-Jyry 2,Kai Lippojoki,2.433,0.100000,4.822936,0.100467,2019-06-15 21:42:07.830285311+00:00,2019-06-15 22:31:21.888686895+00:00,2019-06-15 22:04:19.880065441+00:00
1751,1297,2,Rasti-Jyry 2,Tommi Seitajoki,2.300,0.100000,5.436848,0.071822,2019-06-15 23:19:51.107851744+00:00,2019-06-16 00:24:39.795297146+00:00,2019-06-15 23:49:35.562215328+00:00
1752,1297,3,Rasti-Jyry 2,Mika Ovaskainen,2.132,0.125000,5.833566,0.063766,2019-06-16 01:01:21.535147429+00:00,2019-06-16 02:26:33.572715998+00:00,2019-06-16 01:41:25.035730124+00:00


In [16]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")